In [2]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
import subprocess
import os
import itertools

from tradingbot import TradingBot
from aco_optimiser import Ant

import itertools
import numpy as np

In [3]:
if not os.path.isfile('aco_data.csv'):
    subprocess.run(['python', 'get_data.py'])

data = pd.read_csv("aco_data.csv")


Getting our data file.

In [4]:
def cross_validate(data, window_sizes, acoparams):

    # Load your OHLCV data for the past 720 days
    data_points = len(data)
    train_ratio = 0.8

    # Split the data into train and test sets
    train_size = int(data_points * train_ratio)
    test_size = data_points - train_size

    train_data = data[:train_size]
    test_data = data[train_size:].reset_index(drop=True)

    # Train and validate the ACO model using TimeSeriesSplit
    n_splits = 4

    # 576 training points

    # SPLIT 1: 0 - 175, 176 - 275
    # SPLIT 2: 0 - 275, 276 - 375
    # SPLIT 3: 0 - 375, 376 - 475
    # SPLIT 4: 0 - 475, 476 - 575

    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=100)

    performances = []

    for train_index, val_index in tscv.split(train_data):
        train_split = train_data.loc[train_index].reset_index(drop=True)
        val_split = train_data.loc[val_index].reset_index(drop=True)

        # Train the ACO model on the train_split
        train_bot = TradingBot(train_split, window_sizes, acoparams)
        val_bot = TradingBot(val_split, window_sizes, acoparams)

        train_bot.optimise()
        best_ant = train_bot.best_ants[-1]

        # use top ant to validate
        performances.append(val_bot.run(best_ant.buy_dnf, best_ant.sell_dnf))
    

    return round(sum(performances) / len(performances), 2)


Cross Validate function for a set of window sizes, and acoparams

In [7]:
# Create lists of reasonable values for alpha, beta, evaporation_rate, and mutation_rate
alphas = [1, 2, 3]
betas = [1, 2, 3]
evaporation_rates = [0.1, 0.2, 0.3]

# Choose three window sizes for short, medium, and long periods
window_sizes = [5, 20, 40]

# Optimize alpha, beta, evaporation_rate
best_params = (None, None, None, None)
best_score = -np.inf

for alpha, beta, evaporation_rate in itertools.product(alphas, betas, evaporation_rates):
    print(f"Testing: {alpha}, {beta}, {evaporation_rate}")
    acoparams = (alpha, beta, evaporation_rate, 2, 2, 50, 30)
    score = cross_validate(data, window_sizes, acoparams)

    if score > best_score:
        print(score)
        best_score = score
        best_params = (alpha, beta, evaporation_rate)

print(best_params)
print(best_score)



Testing: 1, 1, 0.1


KeyboardInterrupt: 

Finding optimal alpha, beta and evaporation rate.

In [5]:
best_alpha = 2
best_beta = 1
best_evaporation_rate = 0.2

In [8]:
# Optimize average_literals and average_clauses
best_literals_clauses = (None, None)
best_score_literals_clauses = -np.inf
average_literals = [1, 2, 3, 4, 5, 6]
average_clauses = [1, 2, 3, 4, 5, 6]

for avg_literals, avg_clauses in itertools.product(average_literals, average_clauses):
    print(f"Testing: {avg_literals}, {avg_clauses}")
    acoparams = (best_alpha, best_beta, best_evaporation_rate, avg_literals, avg_clauses, 50, 30)
    score = cross_validate(data, window_sizes, acoparams)

    if score > best_score_literals_clauses:
        print(score)
        best_score_literals_clauses = score
        best_literals_clauses = (avg_literals, avg_clauses)

# Combine the best parameters found
print(best_literals_clauses)
print(best_score_literals_clauses)

Testing: 3, 1
90.61
Testing: 3, 2
Testing: 3, 3
92.67
Testing: 3, 4
Testing: 3, 5
Testing: 3, 6
Testing: 4, 1
Testing: 4, 2
Testing: 4, 3
Testing: 4, 4
95.02
Testing: 4, 5
Testing: 4, 6
Testing: 5, 1
95.29
Testing: 5, 2
Testing: 5, 3
Testing: 5, 4
Testing: 5, 5
Testing: 5, 6
Testing: 6, 1
Testing: 6, 2
Testing: 6, 3
Testing: 6, 4
Testing: 6, 5
Testing: 6, 6
(5, 1)
95.29


Testing: 2, 3
95.8


In [9]:
best_average_literals = 2
best_average_clauses = 3

In [10]:
best_params = (best_alpha, best_beta, best_evaporation_rate, best_average_literals, best_average_clauses, 100, 100)

data_points = len(data)
train_ratio = 0.8

train_size = int(data_points * train_ratio)
test_size = data_points - train_size

train_data = data[:train_size]
test_data = data[train_size:].reset_index(drop=True)


train_bot = TradingBot(train_data, window_sizes, best_params)
test_bot = TradingBot(test_data , window_sizes, best_params)

train_bot.optimise()

best_ant = train_bot.best_ants[-1]
print(test_bot.run(best_ant.buy_dnf, best_ant.sell_dnf))

100


In [24]:
print(best_ant.buy_dnf)
print(best_ant.sell_dnf)

([{('StochasticOscillator_5_40', False), ('StochasticOscillator_20_40', True), ('VortexIndicator_5_20', False), ('UlcerIndex_20_40', False), ('EMAIndicator_20_40', True), ('ChaikinMoneyFlowIndicator_20_40', False)}, {('ForceIndexIndicator_20_40', True), ('BollingerBands_5_40', True), ('DonchianChannel_5_40', False)}], [{('BollingerBands_5_20', False), ('KAMAIndicator_5_20', True)}])
